# Medical Chatbot for Cancer Information Retrieval

This notebook serves as the primary document for the **Cancer Information Retrieval NLP Model**, a retrieval-based Natural Language Processing (NLP) system designed to provide accurate and immediate answers to user queries related to cancer. This project aims to enhance access to trustworthy medical information, alleviate the burden of routine inquiries on healthcare staff, and combat the spread of misinformation often found online.

---

## 1. Business Understanding

This section introduces the real-world problem this project aims to solve, outlines the specific project objectives, and identifies the key stakeholders who would benefit from this solution. 

### 1.1 Problem Statement

The increasing volume of online health information, coupled with the high demand for medical insights and the limited capacity of healthcare professionals, creates a significant challenge. Patients often struggle to find reliable, cancer-specific information quickly, leading to misinformation and increased strain on healthcare support systems.

### 1.2 Project Objectives

The key objectives of this Cancer Information Retrieval NLP Model project are to:

* **Develop a robust text preprocessing pipeline:** Clean and normalize medical text data from the MedQuAD dataset to ensure optimal input for NLP models.
* **Implement an effective retrieval mechanism:** Design and build a system (e.g., using TF-IDF or semantic embeddings) capable of accurately matching user queries to relevant questions/answers in the dataset.
* **Evaluate model performance:** Assess the retrieval model's accuracy and efficiency using appropriate metrics and test cases to ensure it meets the project's goals.
* **Prepare for API integration:** Structure the core NLP model's components to be easily deployable as a backend service for potential chatbot applications.
* **Enhance user experience:** Contribute to a system that provides fast, reliable, and relevant information, ultimately improving how users access cancer-related knowledge.

### 1.3 Stakeholders

The primary stakeholders for this project are:

* **Patients/General Public:** Who need quick, reliable access to cancer information without sifting through unreliable online sources.
* **Healthcare Providers/Hospitals:** Who can integrate this bot backend to streamline their patient support services and reduce manual inquiry handling.
* **Medical Researchers/Educators:** Who can use the structured Q&A data for analysis or educational purposes.

This NLP model is designed to serve as a robust backend for integration into various bot platforms, optimizing information dissemination and combating misinformation.


## 2. Setup and Library Imports

This section handles the necessary library imports and initial setup required for data processing and model development.

In [1]:
#Importing necessary libraries

import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import warnings
import nltk
import re
import os
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
warnings.filterwarnings('ignore')

### 2.1 NLTK Downloads

Some NLTK functionalities require specific datasets to be downloaded. The following cell ensures these are available for text processing tasks like tokenization and lemmatization.

In [2]:
# Download improved NLP resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\biggm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\biggm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\biggm\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\biggm\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

## 3 Data Understanding

### 3.1 Data Loading and Initial Overview

The core of this project relies on the **MedQuAD: Medical Question-Answer Dataset**, which is available on Kaggle. This dataset is a valuable resource containing medical questions and their corresponding expert answers, highly suitable for building a retrieval-based information system.

#### 3.1.1 Custom Data Loader: `CancerQALoader`

To efficiently process the XML structured data from the MedQuAD dataset, a custom Python class `CancerQALoader` has been developed. This class is responsible for:
* Parsing the XML files to extract relevant question-answer pairs.
* Handling potential parsing errors.
* Consolidating data from multiple XML files within a specified folder into a single, clean Pandas DataFrame.

Below is the definition of the `CancerQALoader` class.

In [3]:
class CancerQALoader:
    def __init__(self, filepath):
        self.filepath = filepath
        self.questions = []
        self.answers = []
        self.root = None
        self.source = os.path.splitext(os.path.basename(filepath))[0]

    def parse_xml(self):
        try:
            tree = ET.parse(self.filepath)
            self.root = tree.getroot()
        except ET.ParseError as e:
            print(f"Error parsing XML in {self.filepath}: {e}")
        except FileNotFoundError:
            print(f"File not found: {self.filepath}")

    def extract_qa_pairs(self):
        if self.root is None:
            return

        for qa_pair in self.root.findall('.//QAPair'):
            question_element = qa_pair.find('Question')
            answer_element = qa_pair.find('Answer')

            question = question_element.text if question_element is not None else None
            answer = answer_element.text if answer_element is not None else None

            if question and answer:
                self.questions.append(question)
                self.answers.append(answer)

    def get_dataframe(self):
        if not self.questions or not self.answers:
            return pd.DataFrame(columns=['question', 'answer', 'source'])

        return pd.DataFrame({
            'question': self.questions,
            'answer': self.answers,
            'source': [self.source] * len(self.questions)
        })

    def load_all_qa_from_folder(self, folder_path):
        all_dfs = []
        if not os.path.isdir(folder_path):
            print(f"Error: Folder not found: {folder_path}")
            return pd.DataFrame(columns=['question', 'answer', 'source'])

        xml_files = [f for f in os.listdir(folder_path) if f.endswith(".xml")]
        if not xml_files:
            print(f"Warning: No XML files found in {folder_path}. Please check the folder content.")
            return pd.DataFrame(columns=['question', 'answer', 'source'])

        for filename in xml_files:
            full_path = os.path.join(folder_path, filename)
            loader = CancerQALoader(full_path)
            loader.parse_xml()
            loader.extract_qa_pairs()
            df = loader.get_dataframe()
            if not df.empty:
                all_dfs.append(df)

        if not all_dfs:
            print(f"Warning: XML files found in '{folder_path}' but no Q&A pairs were successfully extracted from any of them. Check XML structure and XPath.")

        return pd.concat(all_dfs, ignore_index=True) if all_dfs else pd.DataFrame(columns=['question', 'answer', 'source'])

In [4]:
folder = "../data/1_CancerGov_QA"

data_loader = CancerQALoader(filepath="dummy_path.xml")

try:
    cancer_df = data_loader.load_all_qa_from_folder(folder)
    
    print(f"Dataset loaded successfully from '{folder}'.")
    print(f"Number of unique source files loaded: {cancer_df['source'].nunique()} files.")
    
except FileNotFoundError:
    print(f"Error: The folder '{folder}' was not found. Please verify the path to your MedQuAD XML files based on your project structure.")
except Exception as e:
    print(f"An error occurred during data loading: {e}")

Dataset loaded successfully from '../data/1_CancerGov_QA'.
Number of unique source files loaded: 116 files.


### 3.2 Dataset Suitability and Feature Description

The MedQuAD dataset, with its structured question-answer pairs extracted from trusted medical sources (like CancerGov.com in this case), is exceptionally well-suited for building a retrieval-based NLP model. Each entry directly provides a query (`question`) and its authoritative response (`answer`), which perfectly aligns with the input-output requirements of a conversational AI backend.

The primary features (columns) extracted and used from the MedQuAD dataset are:

* **`question`**: This column contains the original medical questions. These will serve as the knowledge base against which incoming user queries are compared to find the most relevant match. The inclusion of this feature is justified as it directly provides the query component necessary for a Q&A system.
* **`answer`**: This column contains the detailed, expert-provided answers corresponding to each question. This is the information that will be retrieved and delivered to the user as the system's response. Its inclusion is justified as it provides the authoritative response for the system.
* **`source`**: This column indicates the original medical website (e.g., CancerGov.com) from which the Q&A pair was extracted. Its inclusion is justified as it provides valuable metadata, ensures transparency, and reinforces the trustworthiness of the information provided, which is critical in a medical context.

In [5]:
print("\nDisplaying the first 20 rows of the DataFrame:")
display(cancer_df.head(20))


Displaying the first 20 rows of the DataFrame:


,question,answer,source
0,What is (are) Adult Acute Lymphoblastic Leukem...,Key Points\n - Adult acute ...,0000001_1
1,What are the symptoms of Adult Acute Lymphobla...,"Signs and symptoms of adult ALL include fever,...",0000001_1
2,How to diagnose Adult Acute Lymphoblastic Leuk...,Tests that examine the blood and bone marrow a...,0000001_1
3,What is the outlook for Adult Acute Lymphoblas...,Certain factors affect prognosis (chance of re...,0000001_1
4,Who is at risk for Adult Acute Lymphoblastic L...,Previous chemotherapy and exposure to radiatio...,0000001_1
5,What are the stages of Adult Acute Lymphoblast...,Key Points\n - Once adult A...,0000001_1
6,What are the treatments for Adult Acute Lympho...,Key Points\n - There are di...,0000001_1
7,What is (are) Adult Acute Myeloid Leukemia ?,Key Points\n - Adult acute ...,0000001_2
8,Who is at risk for Adult Acute Myeloid Leukemi...,"Smoking, previous chemotherapy treatment, and ...",0000001_2
9,What are the symptoms of Adult Acute Myeloid L...,"Signs and symptoms of adult AML include fever,...",0000001_2


In [6]:
#Checking the shape of the DataFrame to see how many rows and columns it contains
print(f"\nDataset shape: {cancer_df.shape[0]} rows, {cancer_df.shape[1]} columns")


Dataset shape: 729 rows, 3 columns


In [7]:
#Checking the information about the DataFrame, including data types and non-null counts
print("\nDataset Information:")
cancer_df.info()


Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 729 entries, 0 to 728
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  729 non-null    object
 1   answer    729 non-null    object
 2   source    729 non-null    object
dtypes: object(3)
memory usage: 17.2+ KB


### 3.3 Descriptive Statistics and Data Limitations

A crucial step in data understanding is to examine the dataset's descriptive statistics and identify any inherent limitations. This helps in making informed decisions for subsequent data preparation and modeling.

The dataset consists of `question`, `answer`, and `source` columns. As these are primarily text-based features, traditional numerical descriptive statistics (like mean, std dev) are not directly applicable. Instead, we focus on properties relevant to text data:

* **Dataset Size:** As observed during loading, the dataset contains 729 entries (Q&A pairs) and 3 columns.
* **Unique Sources:** There are 116 distinct source files from which the Q&A pairs were extracted, indicating the breadth of the medical topics covered.
* **Null Values:** We will explicitly check for any missing values in the `question` or `answer` columns, as these would directly impact the model's performance.

**Data Limitations:**
While comprehensive, this dataset might have certain limitations that could impact the project:

* **Specificity of Questions:** The questions are related to "cancer." The model's effectiveness will be limited to this domain; it cannot answer general medical queries.
* **Vocabulary and Jargon:** Medical text often contains highly specialized vocabulary. While necessary for accuracy, this might require robust text preprocessing (e.g., handling acronyms, medical terms not in standard dictionaries) and potentially large vocabulary models.
* **Ambiguity and Nuance:** Natural language, especially in a medical context, can be ambiguous or require nuanced understanding (e.g., distinguishing similar conditions). A retrieval-based model may struggle with highly nuanced or subjective queries.
* **Static Dataset:** As a static dataset, it won't inherently update with new medical discoveries or evolving information. A production system would require mechanisms for periodic data refreshes.
* **Bias in Source Data:** Although from reputable sources, any inherent biases or outdated information present in the original XML files will be reflected in the model's responses.

In [8]:
print("Descriptive Statistics for the Dataset:")

print("Missing values per column:")
display(cancer_df.isnull().sum())

print(f"\nTotal number of Q&A pairs: {cancer_df.shape[0]}")
print(f"Number of unique questions: {cancer_df['question'].nunique()}")
print(f"Number of unique answers: {cancer_df['answer'].nunique()}")
print(f"Number of unique sources: {cancer_df['source'].nunique()}")

print("\nExample Question-Answer Pairs (5 random samples):")
display(cancer_df[['question', 'answer']].sample(5, random_state=42))

Descriptive Statistics for the Dataset:
Missing values per column:


question    0
answer      0
source      0
dtype: int64


Total number of Q&A pairs: 729
Number of unique questions: 683
Number of unique answers: 705
Number of unique sources: 116

Example Question-Answer Pairs (5 random samples):


,question,answer
468,What is the outlook for Metastatic Squamous Ne...,Certain factors affect prognosis (chance of re...
148,How to prevent Anal Cancer ?,Key Points\n - Avoiding ris...
302,What are the stages of Myelodysplastic/ Myelop...,Key Points\n - There is no ...
355,What is the outlook for Extragonadal Germ Cell...,Certain factors affect prognosis (chance of re...
515,What is (are) Pancreatic Neuroendocrine Tumors...,Key Points\n - Pancreatic n...


## 4. Data Cleaning & Preprocessing

This section is dedicated to transforming the raw MedQuAD dataset into a clean and structured format suitable for building a robust NLP model. It involves two primary phases: initial data cleaning (addressing issues like duplicate entries) and detailed text preprocessing (normalizing and preparing the textual content of questions and answers).

### 4.1 Initial Data Cleaning: Handling Duplicates

The first step in ensuring data quality is to identify and address redundant entries. In this dataset, it's crucial to ensure that we don't have multiple identical questions, as this could skew our model's understanding or lead to redundant retrievals. We will check for duplicate questions and remove them, retaining the first occurrence.

In [9]:
#Checking the number of rows in the DataFrame to see how many Q&A pairs were loaded
len(cancer_df)

729

In [10]:
#Checking the data type of the DataFrame to confirm it is a pandas DataFrame
type(cancer_df)

pandas.core.frame.DataFrame

In [11]:
# Check how many duplicate questions exist
duplicate_questions = cancer_df[cancer_df.duplicated(subset='question', keep=False)]
print(f"Total duplicate questions: {duplicate_questions.shape[0]}")
duplicate_questions


Total duplicate questions: 78


,question,answer,source
22,What is (are) Chronic Myelogenous Leukemia ?,Key Points\n - Chronic myel...,0000001_4
29,What are the treatments for Chronic Myelogenou...,Key Points\n - There are di...,0000001_4
246,What is (are) Liver (Hepatocellular) Cancer ?,Key Points\n - Liver cancer...,0000007_4
247,Who is at risk for Liver (Hepatocellular) Canc...,Being infected with certain types of the hepat...,0000007_4
250,What is (are) Liver (Hepatocellular) Cancer ?,Key Points\n - Liver cancer...,0000007_5
...,...,...,...
678,Who is at risk for Prostate Cancer? ?,Different factors increase or decrease the ris...,0000036_3
695,What is (are) Colorectal Cancer ?,Key Points\n - Colorectal c...,0000037_3
697,Who is at risk for Colorectal Cancer? ?,Key Points\n - Avoiding ris...,0000037_3
699,What is (are) Colorectal Cancer ?,Key Points\n - Colorectal c...,0000037_4


In [ ]:
#Removing duplicate questions from the DataFrame while keeping the first occurrence
cancer_df=cancer_df.drop_duplicates(subset='question', keep='first').reset_index(drop=True)

In [ ]:
#Checking the shape of the DataFrame after removing duplicates to see how many rows remain
cancer_df.shape

### 4.2 Text Preprocessing: Normalizing Questions and Answers

After initial cleaning, the textual content of the question and answer columns requires meticulous preprocessing to enhance consistency and relevance for NLP tasks. Medical text often presents unique challenges, such as specialized terminology, boilerplate phrases, and variations in formatting. The following steps standardize the text, making it more digestible for the retrieval model.

#### Defining a Custom Text Cleaning Function (clean_text)
To handle the specific characteristics of medical text, a custom function clean_text is implemented. This function performs a series of transformations:

* Medical-Specific Stopwords: A curated set of stopwords (MEDICAL_STOPWORDS) is defined. These are terms that, while common in English, might not add significant value in the context of specific medical inquiries (e.g., 'patient', 'doctor').

* Boilerplate and Special Character Removal: Regular expressions are used to eliminate common boilerplate phrases (like "Key Points:*-"), standardize newline characters and multiple spaces, and remove any remaining HTML tags.

* Case Normalization and Hyphen Preservation: All text is converted to lowercase to ensure uniformity. Importantly, hyphens within medical terms (e.g., "T-cell") are preserved to maintain the integrity and meaning of these specialized phrases.

* Tokenization: The text is broken down into individual words or subword units (tokens) using word_tokenize.

* Filtering and Medical Plural Handling:Tokens shorter than two characters are generally removed, with exceptions for medical stopwords that might be short but relevant.

* A specialized rule is applied to standardize common medical plural forms (e.g., 'therapies' to 'therapy', 'diseases' to 'disease') to ensure consistency before lemmatization.

* Lemmatization: The WordNetLemmatizer reduces words to their base or dictionary form (e.g., 'running' becomes 'run'), further standardizing the vocabulary.

This multi-stage cleaning approach ensures that the dataset's textual content is refined, removing noise while retaining critical medical context, which is vital for the accuracy of our retrieval system.

In [ ]:
# New temporary cell to inspect data cleaning impact
print("Original Answer Example:")
# Get a random row to inspect, preferably one that was giving jibberish
original_answer_example = cancer_df.iloc[0]['answer'] # Or pick a specific index
print(original_answer_example)

In [ ]:
# Defining a function to clean and preprocess the text data in the DataFrame
MEDICAL_STOPWORDS = set(stopwords.words('english')) | {
    'patient', 'cancer', 'may', 'also', 'include', 'following',
    'key', 'point', 'points', 'doctor', 'medical'
}

lemmatizer = WordNetLemmatizer()

def clean_text(text):
    """
    Improved medical text cleaning with boilerplate removal and specialized handling.
    """
    if not isinstance(text, str):
        return ""

    # 1. Remove all boilerplate patterns, newlines, and collapse multiple spaces
    text = re.sub(r'Key Points.*?-', '', text)
    text = re.sub(r'\\n|\n', ' ', text) # Replace newlines with spaces
    text = re.sub(r'\s+', ' ', text)    # Collapse multiple spaces

    # 2. Remove HTML tags and keep hyphens in medical terms, then convert to lowercase
    text = re.sub(r'<.*?>', '', text)   # Remove HTML tags
    text = re.sub(r'[^a-zA-Z0-9\s-]', '', text).lower() # Keep hyphens in terms

    # 3. Enhanced medical token handling: tokenization, filtering, medical plural handling, and lemmatization
    tokens = word_tokenize(text)
    filtered_tokens = []

    for word in tokens:
        # Keep medical terms longer than 2 chars, except stopwords
        if len(word) > 2 and (word not in MEDICAL_STOPWORDS):
            # Special handling for medical plurals (e.g., therapies -> therapy)
            lemma = lemmatizer.lemmatize(word, pos='n')
            if lemma.endswith('ies'):
                lemma = lemma[:-3] + 'y' # e.g., therapies -> therapy
            elif lemma.endswith('es'):
                lemma = lemma[:-2] # e.g., diseases -> disease (simple case)
            filtered_tokens.append(lemma)

    return ' '.join(filtered_tokens)


In [ ]:
print("\nCleaned Answer Example (after current preprocessing):")
# Assuming clean_text was applied like this:
# cancer_df['answer'] = cancer_df['answer'].apply(clean_text)
# If it wasn't, you need to explicitly clean it here for testing:
cleaned_answer_example = clean_text(original_answer_example)
print(cleaned_answer_example)

#### Applying Preprocessing to the Dataset
The clean_text function is now applied to both the question and answer columns of our cancer_df DataFrame. This ensures that all text data used for subsequent NLP tasks is consistently cleaned and normalized.

In [ ]:
#remove spaces and new lines
cancer_df['question'] = cancer_df['question'].str.strip().replace('\n', ' ')
cancer_df['answer_display'] = cancer_df['answer'].str.strip().replace('\n', ' ')


# Preprocess the questions and answers
cancer_df['question'] = cancer_df['question'].apply(clean_text)
# cancer_df['answer'] = cancer_df['answer'].apply(clean_text)


#### Verifying Preprocessing Results
After applying the preprocessing steps, it's important to verify the changes. We'll re-examine the number of unique answers (which might change slightly due to normalization) and display the first few rows of the DataFrame to visually inspect the cleaned text

In [ ]:
#Checking the number of unique answers in the DataFrame
print(f"Number of unique answers: {cancer_df['answer'].nunique()}")
print(cancer_df['answer'].value_counts().head(5))


In [ ]:
#Displaying the first 10 rows of the DataFrame
cancer_df.head(10)

## 5. Hybrid Search System

To build a robust cancer information retrieval system, a hybrid approach leveraging both keyword-based and semantic search capabilities will be implemented. This strategy aims to combine the precision of traditional term matching with the nuanced understanding of modern deep learning models, thereby improving the relevance and comprehensiveness of search results for diverse user queries.

### 5.1 Keyword-Based Retrieval: TF-IDF Vectorization

Term Frequency-Inverse Document Frequency (TF-IDF) is a statistical measure that evaluates how relevant a word is to a document in a collection of documents. It's particularly effective for keyword-based matching and identifying important terms within questions.

For our hybrid system, TF-IDF will be used to create sparse vector representations of the `question` column. These vectors will serve as the basis for a keyword-driven retrieval component, allowing us to find questions with significant term overlap with a user's query. The `clean_text` preprocessor is integrated directly to ensure the TF-IDF vectors are generated from normalized text.

In [ ]:
# Initialize models
tfidf_vectorizer = TfidfVectorizer(preprocessor=clean_text, stop_words=list(MEDICAL_STOPWORDS))
X_tfidf = tfidf_vectorizer.fit_transform(cancer_df['question'])

### 5.2 Semantic Retrieval: BioBERT Question Embeddings
Beyond keyword matching, understanding the semantic meaning of questions is crucial for handling rephrased queries or those using synonyms. For this, we utilize a pre-trained BioBERT (Bidirectional Encoder Representations from Transformers for Biomedical Text Mining) model. BioBERT is a specialized BERT model trained on extensive biomedical text corpora, making it highly effective at generating contextually rich, dense vector embeddings (sentence embeddings) for medical queries.

These embeddings capture the nuanced meaning of questions and will enable our system to retrieve answers based on semantic similarity, even if exact keywords are not present.

In [ ]:
# Biomedical embedding model
model = SentenceTransformer("pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb")
question_embeddings = model.encode(cancer_df['question'].tolist())

### 5.3 2D Visualization of BioBERT Question Embeddings
Visualizing high-dimensional embeddings can provide valuable insights into the quality of the representations and reveal underlying structures or clusters within the data. We will use t-Distributed Stochastic Neighbor Embedding (t-SNE) for dimensionality reduction, projecting the BioBERT question embeddings from their high-dimensional space into a 2D plot.

The t-SNE plot helps to:

* Identify potential clusters of semantically similar questions.

* Verify if related questions are grouped together.

* Gain an intuitive understanding of the embedding space.

In [ ]:
# 2D Visualization of Sentence Embeddings
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

tsne = TSNE(n_components=2, random_state=42)
embeddings_2d = tsne.fit_transform(question_embeddings)

plt.figure(figsize= (6,5))
plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], alpha=0.6)
plt.title("2D Visualization of BioBERT Question Embeddings")
plt.xlabel("Component 1")
plt.ylabel("Component 2")
plt.grid(True)
plt.show()

The scatter plot above displays a **2D t-SNE projection** of semantic embeddings generated by the BioBERT-based SentenceTransformer model for each cancer-related question in the dataset.
#### Key Observations:
- Several **tight clusters** can be seen, suggesting that BioBERT effectively groups similar medical questions like those about:
  - Symptoms and diagnosis
  - Cancer types
  - Treatment and medications
  - Survival and prognosis
- **Scattered points** may indicate outliers or uniquely phrased questions that do not fit well into larger semantic groups.

## 6. Hybrid Search Function

With the text data preprocessed and both TF-IDF and BioBERT semantic embeddings generated, the next critical step is to develop the **hybrid search function**. This function will serve as the core of our chatbot's retrieval mechanism, taking a user's query and returning the most relevant question-answer pairs from our dataset.

The `hybrid_search` function combines the strengths of keyword-based matching (using TF-IDF) and semantic understanding (using BioBERT embeddings). It allows for tunable weights to balance the influence of each component and includes a confidence threshold and an optional disease-specific filter for more precise results.

In [ ]:
def hybrid_search(query, n=3, tfidf_weight=0.0, semantic_weight=1.0, threshold=0.30, disease_filter=None):
    """Enhanced hybrid search with disease filtering"""
    # Process query
    clean_query = clean_text(query)
    
    # TF-IDF similarity
    tfidf_vec = tfidf_vectorizer.transform([clean_query])
    tfidf_sim = cosine_similarity(tfidf_vec, X_tfidf).flatten()
    
    # Semantic similarity
    semantic_vec = model.encode([query])
    semantic_sim = cosine_similarity(semantic_vec, question_embeddings).flatten()
    
    # Combined score
    combined_sim = (tfidf_weight * tfidf_sim) + (semantic_weight * semantic_sim)
    
    # Get top results
    top_indices = np.argsort(combined_sim)[-n*3:][::-1]  # Get extra results for filtering
    top_matches = []
    for idx in top_indices:
        if combined_sim[idx] > threshold:
            match = {
                'question': cancer_df.iloc[idx]['question'],
                'answer': cancer_df.iloc[idx]['answer_display'],
                'similarity': float(combined_sim[idx]),
                'type': 'TF-IDF' if tfidf_sim[idx] > semantic_sim[idx] else 'Semantic',
                'tfidf_sim': float(tfidf_sim[idx]),       # <-- ADD THIS LINE
                'semantic_sim': float(semantic_sim[idx]) # <-- AND THIS LINE
            }
            
            # Apply disease filter if provided
            if not disease_filter or disease_filter.lower() in match['question'].lower():
                top_matches.append(match)
                
            if len(top_matches) >= n:  # Stop when we have enough filtered results
                break
    if not top_matches:
        return [{
            'question': "No confident match found", # Added
            'answer': "I couldn't find a confident match. Please consult a doctor or rephrase your question.",
            'similarity': 0.0,
            'type': 'N/A', # Added
            'tfidf_sim': 0.0, # Added
            'semantic_sim': 0.0 # Added
        }]
    
    return top_matches[:n]  # Return exactly n results

## 7. Model Evaluation

To quantitatively assess the effectiveness of the hybrid search system, a dedicated evaluation framework is implemented. This framework uses a `validation_set` containing predefined queries, expected keywords, and specific testing parameters like `disease` filters and `min_similarity` thresholds.

The `evaluate_model` function systematically runs each test case from the `validation_set` through the `hybrid_search` function. It then analyzes the retrieved answers to determine if any of the `expected` keywords are present. This provides an indication of whether the system successfully retrieves relevant information.

The evaluation process includes:
* **Structured Test Cases**: A `validation_data` list defines specific queries with associated `expected` keywords that should appear in a relevant answer, an optional `disease` filter, and a `min_similarity` threshold for that query.
* **Hybrid Search Execution**: Each query is passed to the `hybrid_search` function, utilizing the `disease` filter and `min_similarity` for the test.
* **Keyword Presence Check**: The function checks if any of the `expected` keywords are present in the `answer` of the top retrieved results.
* **Pass/Fail Determination**: A test case is marked as "PASS" if at least one expected keyword is found in any of the retrieved answers, indicating a relevant match.
* **Detailed Logging**: Comprehensive print statements provide immediate feedback on each test case, including found keywords, best match details, and the overall result (PASS/FAIL).
* **Summary Statistics**: A final report provides the total number of tests, passed tests, failed tests, and an overall accuracy score.
* **Failed Case Details**: An optional section displays more detailed information for any test cases that failed.

This evaluation approach allows for targeted assessment of the model's ability to retrieve information for specific types of cancer-related queries and helps in identifying areas for improvement.

In [ ]:
# Cell 1: Enhanced validation set with disease-specific testing
validation_data = [
    {
        "query": "prostate cancer symptoms", 
        "expected": ["urinary", "urination", "flow", "frequent", "weak stream"],
        "disease": "prostate",
        "min_similarity": 0.5
    },
    {
        "query": "breast cancer treatment options", 
        "expected": ["surgery", "radiation", "chemotherapy", "mastectomy", "lumpectomy"],
        "disease": "breast",
        "min_similarity": 0.45
    },
    {
        "query": "leukemia causes", 
        "expected": ["genetic", "mutation", "radiation", "chemical", "chromosome"],
        "disease": "leukemia",
        "min_similarity": 0.4
    },
    {
        "query": "What are the common symptoms of lung cancer?",
        "expected": ["cough", "chest pain", "shortness of breath", "weight loss"],
        "disease": "lung",
        "min_similarity": 0.4
    },
    {
        "query": "How is melanoma diagnosed?",
        "expected": ["biopsy", "skin exam", "imaging"],
        "disease": "melanoma",
        "min_similarity": 0.45
    },
    {
        "query": "Can smoking cause cancer?",
        "expected": ["risk factor", "lung", "esophagus", "mouth", "throat"],
        "disease": None, # No specific disease filter for general cancer query
        "min_similarity": 0.35 # Might expect lower similarity for very broad questions
    },
    {
        "query": "What is the best way to cook pasta?", # Negative test case
        "expected": [], # No expected keywords from cancer dataset
        "disease": None,
        "min_similarity": 0.1 # Very low threshold to ensure it doesn't match
    }
]

In [ ]:
# Cell 2: evaluate_model function definition

def evaluate_model(validation_set, n_results=1, tfidf_weight=0.4, semantic_weight=0.6):

    total_tests = len(validation_set)
    passed_tests_count = 0
    detailed_evaluation_results = [] # To store detailed results for programmatic analysis

    print("=" * 80)
    print("MEDICAL CHATBOT EVALUATION REPORT")
    print("=" * 80)

    for i, test in enumerate(validation_set):
        query = test["query"]
        expected_keywords = [k.lower() for k in test["expected"]] # Ensure expected keywords are lowercase
        disease_filter = test.get("disease")
        min_similarity_threshold = test.get("min_similarity", 0.35) # Use a reasonable default

        print(f"\nTEST CASE {i+1}: {query}")
        print(f"DISEASE FILTER: {disease_filter if disease_filter else 'None'}")
        print(f"EVALUATION THRESHOLD: {min_similarity_threshold:.2f}")
        print(f"USING HYBRID WEIGHTS: TF-IDF={tfidf_weight:.2f}, Semantic={semantic_weight:.2f}")
        print("-" * 60)

        # Call hybrid_search with the specified weights for evaluation
        # We set threshold=0.0 in hybrid_search to always get 'n_results'
        # and then apply min_similarity_threshold manually for evaluation logic.
        results_from_hybrid_search = hybrid_search(
            query,
            n=n_results,
            tfidf_weight=tfidf_weight,
            semantic_weight=semantic_weight,
            threshold=0.0 # Temporarily set threshold low to get top N results for analysis
        )
        
        # Initialize default values for results
        test_passed = False
        found_keywords_in_top_match = []
        top_match_info = {
            "question": "No confident match found or below threshold", # More specific message
            "answer": "N/A",
            "similarity": 0.0,
            "type": "N/A",
            "tfidf_sim": 0.0,
            "semantic_sim": 0.0
        }

        if results_from_hybrid_search:
            # The top match is the first one, as hybrid_search sorts by similarity
            current_top_match = results_from_hybrid_search[0]
            
            top_match_info['question'] = current_top_match['question']
            top_match_info['answer'] = current_top_match['answer']
            top_match_info['similarity'] = current_top_match['similarity']
            top_match_info['type'] = current_top_match['type']
            top_match_info['tfidf_sim'] = current_top_match.get('tfidf_sim', 0.0) # Use .get() with default
            top_match_info['semantic_sim'] = current_top_match.get('semantic_sim', 0.0) # Use .get() with default

            # Check if the top match meets the min_similarity_threshold for this test case
            if top_match_info['similarity'] >= min_similarity_threshold:
                # Check for expected keywords in the answer of the top match
                answer_text_lower = clean_text(top_match_info['answer']).lower() # Ensure consistent cleaning
                for keyword in expected_keywords:
                    if keyword in answer_text_lower: 
                        found_keywords_in_top_match.append(keyword)
                
                # A test passes if it meets the similarity threshold AND finds AT LEAST ONE expected keyword
                # If expected_keywords is empty (e.g., for negative tests), it passes if similarity is low.
                if (found_keywords_in_top_match) or (not expected_keywords and top_match_info['similarity'] < min_similarity_threshold + 0.1): 
                    test_passed = True
            elif not expected_keywords and top_match_info['similarity'] < min_similarity_threshold:
                # Special case: If no keywords are expected (negative test) AND similarity is low, it passes
                test_passed = True


        print(f"EXPECTED KEYWORDS: {', '.join(expected_keywords) if expected_keywords else 'None'}")
        print(f"FOUND KEYWORDS (in top match): {', '.join(found_keywords_in_top_match) if found_keywords_in_top_match else 'None'}")
        print(f"TOP MATCH SIMILARITY: {top_match_info['similarity']:.2f}")
        print(f"TOP MATCH TYPE: {top_match_info['type']}")
        
        # Print individual similarities if available
        if top_match_info['tfidf_sim'] != 0.0 or top_match_info['semantic_sim'] != 0.0:
            print(f"  (TF-IDF Sim: {top_match_info['tfidf_sim']:.2f}, Semantic Sim: {top_match_info['semantic_sim']:.2f})")

        print(f"BEST MATCH QUESTION: {top_match_info['question'][:100]}...") # Truncate for display
        print(f"BEST MATCH ANSWER: {top_match_info['answer'][:200]}...") # Truncate for display
        
        result_status = "PASS" if test_passed else "FAIL"
        print(f"RESULT: {result_status}")

        if test_passed:
            passed_tests_count += 1
        
        # Store comprehensive results for each test case
        detailed_evaluation_results.append({
            "query": query,
            "passed": test_passed,
            "expected_keywords": expected_keywords,
            "found_keywords": found_keywords_in_top_match,
            "top_match_info": top_match_info,
            "evaluation_threshold": min_similarity_threshold,
            "used_tfidf_weight": tfidf_weight,
            "used_semantic_weight": semantic_weight
        })

    print("\n" + "=" * 80)
    print("SUMMARY")
    print("=" * 80)
    print(f"TOTAL TESTS: {total_tests}")
    print(f"PASSED: {passed_tests_count}")
    print(f"FAILED: {total_tests - passed_tests_count}")
    accuracy = (passed_tests_count / total_tests) * 100 if total_tests > 0 else 0
    print(f"ACCURACY: {accuracy:.2f}%")
    print("=" * 80)

    print("\nFAILED CASE DETAILS:")
    failed_cases_found = False
    for res in detailed_evaluation_results:
        if not res['passed']:
            failed_cases_found = True
            print(f"\nQuery: {res['query']}")
            print(f"  Expected Keywords: {', '.join(res['expected_keywords'])}")
            print(f"  Found Keywords: {', '.join(res['found_keywords']) if res['found_keywords'] else 'None'}")
            print(f"  Top Match Similarity: {res['top_match_info']['similarity']:.2f} (Threshold: {res['evaluation_threshold']:.2f})")
            print(f"  Closest Match Question: {res['top_match_info']['question']}")
            print(f"  Match Answer (partial): {res['top_match_info']['answer'][:200]}...")
            
            reason_for_failure = []
            if res['top_match_info']['similarity'] < res['evaluation_threshold']:
                reason_for_failure.append("Similarity below evaluation threshold")
            if res['expected_keywords'] and not res['found_keywords']:
                reason_for_failure.append("Missing expected keywords")
            # If it's a negative test, and it still got a high score
            if not res['expected_keywords'] and res['top_match_info']['similarity'] >= res['evaluation_threshold']:
                reason_for_failure.append("Unexpected high similarity for negative test")

            print(f"  Reason for Failure: {'; '.join(reason_for_failure) if reason_for_failure else 'Unknown'}")
    if not failed_cases_found:
        print("None")
        
    return detailed_evaluation_results

In [ ]:
# Cell 3: Run the evaluation
# This will call the evaluate_model function and automatically print the summary and failed cases.
evaluation_results = evaluate_model(validation_data)

### Model Tuning

In [ ]:
# Cell for Model Initialization
# This initializes your TF-IDF vectorizer and BioBERT model embeddings.
# Make sure 'clean_text' function and 'cancer_df' are already defined and loaded.

print("Initializing TF-IDF Vectorizer...")
tfidf_vectorizer = TfidfVectorizer(preprocessor=clean_text, stop_words=list(MEDICAL_STOPWORDS))
X_tfidf = tfidf_vectorizer.fit_transform(cancer_df['question'])
print(f"TF-IDF Vectorizer fitted. Shape of X_tfidf: {X_tfidf.shape}")

print("\nInitializing BioBERT Embedding Model (this may take a moment)...")
# Ensure 'SentenceTransformer' is imported from 'sentence_transformers'
# If you get a ModuleNotFoundError, ensure you've installed it: pip install sentence-transformers
model = SentenceTransformer("pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb")
question_embeddings = model.encode(cancer_df['question'].tolist())
print(f"BioBERT model loaded. Shape of question_embeddings: {question_embeddings.shape}")
print("TF-IDF and BioBERT models initialized successfully.")

### Weight Tuning Experiment

This section systematically tunes the contribution of TF-IDF (lexical similarity) and BioBERT embeddings (semantic similarity) within our `hybrid_search` function. The goal is to find the optimal combination of weights (`tfidf_weight` and `semantic_weight`) that maximizes the chatbot's accuracy on the predefined `validation_data`.

In [ ]:
# Cell for Weight Tuning

# Define a range of weights to test. They should sum to 1.0.
# np.arange(0.0, 1.05, 0.1) creates values like 0.0, 0.1, 0.2, ..., 1.0
weight_combinations = [(round(w, 2), round(1 - w, 2)) for w in np.arange(0.0, 1.05, 0.1)]

best_accuracy = -1
best_weights = (0.0, 0.0)
best_evaluation_results = None # To store detailed results of the best run

print("=" * 80)
print("STARTING WEIGHT TUNING EXPERIMENT")
print("=" * 80)

for tfidf_w, semantic_w in weight_combinations:
    print(f"\n--- Testing Weights: TF-IDF={tfidf_w:.2f}, Semantic={semantic_w:.2f} ---")
    
    # Call the evaluate_model function with the current weights
    # The evaluate_model will print its own detailed report for each run
    current_evaluation_results = evaluate_model(
        validation_data,
        tfidf_weight=tfidf_w,
        semantic_weight=semantic_w
    )
    
    # Calculate accuracy from the returned results to find the best combination
    # The 'passed' key in each result dictionary indicates if that test case passed.
    current_passed_count = sum(1 for res in current_evaluation_results if res['passed'])
    current_accuracy = (current_passed_count / len(validation_data)) * 100 if len(validation_data) > 0 else 0

    print(f"\nOverall Accuracy for TF-IDF={tfidf_w:.2f}, Semantic={semantic_w:.2f}: {current_accuracy:.2f}%")

    if current_accuracy > best_accuracy:
        best_accuracy = current_accuracy
        best_weights = (tfidf_w, semantic_w)
        best_evaluation_results = current_evaluation_results # Store the full results for detailed analysis later

print("\n" + "=" * 80)
print("WEIGHT TUNING SUMMARY")
print("=" * 80)
print(f"BEST OVERALL ACCURACY: {best_accuracy:.2f}%")
print(f"OPTIMAL WEIGHTS FOUND: TF-IDF={best_weights[0]:.2f}, Semantic={best_weights[1]:.2f}")
print("=" * 80)

# Optional: Display FAILED cases from the BEST performing weight combination
print("\nDETAILED FAILED CASES FOR OPTIMAL WEIGHTS:")
failed_cases_found_optimal = False
if best_evaluation_results:
    for res in best_evaluation_results:
        if not res['passed']:
            failed_cases_found_optimal = True
            print(f"\nQuery: {res['query']}")
            print(f"  Expected Keywords: {', '.join(res['expected_keywords']) if res['expected_keywords'] else 'None'}")
            print(f"  Found Keywords: {', '.join(res['found_keywords']) if res['found_keywords'] else 'None'}")
            print(f"  Top Match Similarity: {res['top_match_info']['similarity']:.2f} (Threshold: {res['evaluation_threshold']:.2f})")
            print(f"  Closest Match Question: {res['top_match_info']['question']}")
            print(f"  Match Answer (partial): {res['top_match_info']['answer'][:200]}...")
            
            reason_for_failure = []
            if res['top_match_info']['similarity'] < res['evaluation_threshold']:
                reason_for_failure.append("Similarity below evaluation threshold")
            if res['expected_keywords'] and not res['found_keywords']:
                reason_for_failure.append("Missing expected keywords")
            if not res['expected_keywords'] and res['top_match_info']['similarity'] >= res['evaluation_threshold']:
                reason_for_failure.append("Unexpected high similarity for negative test")
            print(f"  Reason for Failure: {'; '.join(reason_for_failure) if reason_for_failure else 'Unknown'}")
if not failed_cases_found_optimal:
    print("None")

### Hybrid Search Internal Threshold Tuning

This section focuses on optimizing the `threshold` parameter within the `hybrid_search` function itself. This threshold determines the minimum combined similarity score a match must achieve to be returned by `hybrid_search`. Finding the optimal threshold is crucial for filtering out irrelevant results (especially for negative test cases) while retaining all highly relevant ones. This tuning is performed using the `best_weights` identified in the previous step.

In [ ]:
# Cell for Hybrid Search Internal Threshold Tuning

threshold_range = np.arange(0.0, 1.05, 0.05) # Test thresholds from 0.0 to 1.0 (inclusive)
best_threshold_for_hybrid_search = 0.0
highest_accuracy_for_threshold = -1.0

print("=" * 80)
print("STARTING HYBRID SEARCH INTERNAL THRESHOLD TUNING")
print("=" * 80)

# Use the best weights found from the previous tuning step
# If `best_weights` is not defined (e.g., if you run this cell alone),
# uncomment and set these manually based on your weight tuning results.
current_tfidf_weight, current_semantic_weight = best_weights 
print(f"Using optimal weights for threshold tuning: TF-IDF={current_tfidf_weight:.2f}, Semantic={current_semantic_weight:.2f}")

threshold_accuracies_for_hybrid_search = {}

for threshold_val in threshold_range:
    current_passed_count_threshold = 0
    current_total_tests_threshold = len(validation_data)
    
    for test_case in validation_data:
        # Call hybrid_search with the current threshold_val and the optimal weights
        # Here, `n=1` is typically sufficient as evaluate_model usually only considers the top match.
        temp_results = hybrid_search(
            test_case['query'],
            n=1,
            tfidf_weight=current_tfidf_weight,
            semantic_weight=current_semantic_weight,
            threshold=threshold_val, # THIS IS WHERE THE `hybrid_search`'s internal threshold is set
            disease_filter=test_case.get('disease')
        )

        # Apply the evaluation logic from evaluate_model based on the results from hybrid_search
        is_passed_threshold = False
        if temp_results: # If hybrid_search returned any result above its internal threshold
            top_match = temp_results[0]
            # Check against the test case's specific min_similarity_threshold
            if top_match['similarity'] >= test_case['min_similarity']:
                answer_text_lower = clean_text(top_match['answer']).lower()
                found_keywords = [k for k in test_case['expected'] if k.lower() in answer_text_lower]
                if found_keywords or (not test_case['expected'] and top_match['similarity'] < test_case['min_similarity'] + 0.1):
                    is_passed_threshold = True
            # Special case for negative tests: if no keywords expected and low similarity, it passes
            elif not test_case['expected'] and top_match['similarity'] < test_case['min_similarity']:
                 is_passed_threshold = True
        else: # If hybrid_search returned *no results* (because all were below `threshold_val`)
            # For negative test cases where no result is expected, if hybrid_search returns nothing, it's a pass.
            if not test_case['expected']:
                is_passed_threshold = True
        
        if is_passed_threshold:
            current_passed_count_threshold += 1

    current_accuracy_threshold = (current_passed_count_threshold / current_total_tests_threshold) * 100
    threshold_accuracies_for_hybrid_search[f"Threshold {threshold_val:.2f}"] = current_accuracy_threshold
    
    if current_accuracy_threshold > highest_accuracy_for_threshold:
        highest_accuracy_for_threshold = current_accuracy_threshold
        best_threshold_for_hybrid_search = threshold_val

    print(f"  Accuracy for Threshold {threshold_val:.2f}: {current_accuracy_threshold:.2f}%")

print("\n" + "=" * 80)
print("HYBRID SEARCH INTERNAL THRESHOLD TUNING SUMMARY")
print("=" * 80)
print(f"BEST ACCURACY: {highest_accuracy_for_threshold:.2f}%")
print(f"OPTIMAL HYBRID SEARCH INTERNAL THRESHOLD: {best_threshold_for_hybrid_search:.2f}")
print("=" * 80)

# After identifying the optimal internal threshold, you would usually update the
# `threshold` parameter's default value in your `hybrid_search` function definition.
# e.g., def hybrid_search(query, n=3, tfidf_weight=0.4, semantic_weight=0.6, threshold=YOUR_OPTIMAL_THRESHOLD_HERE):

In [ ]:
# Evaluating the model's accuracy on a validation set

correct = 0
total = len(validation_data)
k = 3  # Top-k

for item in validation_data:
    query = item["query"]
    expected_keywords = item["expected"]

    top_results = hybrid_search(query, n=k, threshold=0.0)

    found = any(
        any(keyword.lower() in result["answer"].lower() for keyword in expected_keywords)
        for result in top_results
    )

    if found:
        correct += 1

accuracy = correct / total
print(f"Top-{k} Accuracy: {accuracy:.2%}")

In [ ]:
# Function to get a response based on user input with a similarity threshold
from difflib import SequenceMatcher

def get_response(user_input, threshold=0.7):
    processed_input = clean_text(user_input)
    for index, row in cancer_df.iterrows():
        question = clean_text(row['question'])
        similarity = SequenceMatcher(None, processed_input, question).ratio()
        if similarity >= threshold:
            return row['answer']
    return "I'm sorry, I don't have information on that. Please consult a doctor."

In [ ]:
for item in hybrid_search("Signs of throat cancer?"):
    print(item["answer"])


### User Interface

This section sets up an interactive web interface using Gradio, allowing users to query the optimized medical chatbot.

In [ ]:
import gradio as gr

def chatbot(query):
    results = hybrid_search(query)
    top_result = results[0]
    if top_result["question"] is None:
        return top_result["answer"]
    return f"*Answer: {top_result['answer']}\nSimilarity*: {top_result['similarity']:.2f}"

interface = gr.Interface(
    fn=chatbot,
    inputs=gr.Textbox(label="Enter your question about cancer"),
    outputs=gr.Textbox(label="Answer"),
    title="Cancer QA Chatbot",
    description="Ask questions about cancer symptoms, treatments, or risk factors."
)
interface.launch()

## Conclusion

Through dedicated effort and a systematic approach, we have successfully developed and optimized a robust medical chatbot designed for cancer information retrieval. Our system stands on a hybrid search methodology, intelligently combining the strengths of traditional TF-IDF with the advanced semantic understanding of BioBERT embeddings.

## Challenges Encountered

Throughout the development of our medical chatbot for cancer information retrieval, we encountered several key challenges. Overcoming these obstacles was crucial for the successful completion and optimization of the system.

1.  **Data Quality and Preprocessing:**
    * **Heterogeneous Data Formats:** The initial MedQuAD dataset, while rich in content, presented in XML format required careful parsing to extract the relevant question-answer pairs and associated metadata. Ensuring consistent extraction across diverse XML structures was an initial hurdle.
    * **Text Cleaning Complexity:** Medical text often contains specific terminology, abbreviations, and formatting that required tailored preprocessing. Striking the right balance with our `clean_text` function was challenging; aggressive cleaning for search (e.g., extensive lemmatization, stopword removal) could lead to unreadable output for display, while insufficient cleaning hindered retrieval accuracy. This was evident when our initial Gradio output was "jibberish," prompting us to implement a dual-cleaning strategy (one for search, one for display). 
    * **Duplicate and Near-Duplicate Entries:** Identifying and handling duplicate or near-duplicate question-answer pairs was essential to prevent bias in our index and ensure unique, meaningful responses.

2.  **Model Selection and Integration:**
    * **Hybrid Search Complexity:** Combining two distinct retrieval methodologies—TF-IDF for lexical matching and BioBERT for semantic understanding—required careful design. Determining how to effectively blend their scores to leverage the strengths of both was a core challenge.
    * **Computational Resources for Embeddings:** Encoding the entire dataset's questions into BioBERT embeddings was a computationally intensive task, especially in environments with limited GPU access. This necessitated efficient batch processing and careful memory management.

3.  **Performance Tuning and Evaluation:**
    * **Defining "Success":** Establishing clear and measurable criteria for what constitutes a "successful" retrieval was more complex than a simple exact match. Our `evaluate_model` function needed to account for similarity thresholds *and* the presence of expected keywords, particularly for distinguishing relevant from irrelevant answers in negative test cases.
    * **Parameter Optimization:** Systematically tuning the `tfidf_weight`, `semantic_weight`, and the internal `threshold` of the `hybrid_search` function was an iterative process. Finding the optimal combination that maximized accuracy across diverse test cases (including negative ones) required careful experimentation and analysis of the evaluation results.
    * **Handling Edge Cases/Negative Tests:** The "What is the best way to cook pasta?" query highlighted a specific challenge: ensuring genuinely irrelevant queries resulted in either no match or a similarity score below the acceptable threshold. Our initial setup struggled with this, as even semantically unrelated texts could show minor (but still problematic) similarity, leading to "Unexpected high similarity for negative test" failures.
4.  **User Interface Development:**
    * **Seamless Integration:** Connecting the backend `hybrid_search` logic to a frontend interface (Gradio) required careful attention to function signatures and data flow. Ensuring that the output from the retrieval system was correctly formatted and displayed in a user-friendly manner was essential for a good user experience.
    * **Displaying Clean vs. Raw Text:** The conflict between text optimized for retrieval (heavily cleaned) and text suitable for human readability (lighter cleaning) for display purposes was a practical challenge that we addressed by maintaining separate `question_cleaned_for_search` and `answer_display` columns. 
    
By systematically addressing these challenges, we were able to refine our approach, improve our code, and ultimately deliver a robust and effective medical chatbot.

## Recommendations for Future Improvements

While our current model performs exceptionally well on the validation set, we have identified several areas for future improvements to further enhance its capabilities and prepare it for real-world deployment:

1.  **Expand and Diversify Training Data:**
    * **Larger Dataset:** We recommend incorporating a larger and more diverse dataset. This would cover a broader range of cancer types, related questions, and comprehensive answers, which would significantly improve the model's generalization.
    * **Data Annotation:** We believe that manually annotating more data with specific entities (e.g., drug names, symptoms) or intent types would enable us to explore more advanced NLP techniques.
    * **Negative Sampling:** Actively curating more diverse negative examples, beyond simple irrelevant queries (like "cook pasta"), to include questions somewhat related to the medical domain but not specific to cancer, would greatly improve the model's robustness.

2.  **Explore Advanced NLP Models:**
    * **Fine-tuning BioBERT/other LLMs:** Instead of solely relying on pre-trained embeddings, fine-tuning BioBERT or other domain-specific Language Models (LLMs) directly on our specific Q&A dataset could further boost performance and understanding of nuances in cancer questions.
    * **Generative AI Integration:** For queries where a direct answer isn't immediately found, we could integrate a generative AI model (e.g., a fine-tuned LLM) capable of synthesizing information from retrieved passages to create more conversational and complete answers.

3.  **Refine Answer Extraction and Summarization:**
    * **Passage Re-ranking:** For the top `n` results retrieved by `hybrid_search`, we could apply a more sophisticated re-ranking model (e.g., a cross-encoder model) to identify the absolute best match, especially when `n` is greater than 1.
    * **Answer Span Extraction:** Instead of returning entire answer documents, we aim to implement techniques to extract only the most relevant sentence or paragraph that directly answers the user's query, providing more concise responses.
    * **Summarization:** For very long answers, we recommend exploring abstractive or extractive summarization techniques to provide users with a brief, digestible overview.